In [1]:
import pandas as pd
data = pd.read_csv('data/mortality_fetal_prediction.xlsx - data.csv')
display(data)
data.info()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120,0.000,0.000,0.000,0.000,0.0,0.0,73,0.5,43,2.4,64,62,126,2,0,120,137,121,73,1,2
1,132,0.006,0.000,0.006,0.003,0.0,0.0,17,2.1,0,10.4,130,68,198,6,1,141,136,140,12,0,1
2,133,0.003,0.000,0.008,0.003,0.0,0.0,16,2.1,0,13.4,130,68,198,5,1,141,135,138,13,0,1
3,134,0.003,0.000,0.008,0.003,0.0,0.0,16,2.4,0,23.0,117,53,170,11,0,137,134,137,13,1,1
4,132,0.007,0.000,0.008,0.000,0.0,0.0,16,2.4,0,19.9,117,53,170,9,0,137,136,138,11,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140,0.000,0.000,0.007,0.000,0.0,0.0,79,0.2,25,7.2,40,137,177,4,0,153,150,152,2,0,2
2122,140,0.001,0.000,0.007,0.000,0.0,0.0,78,0.4,22,7.1,66,103,169,6,0,152,148,151,3,1,2
2123,140,0.001,0.000,0.007,0.000,0.0,0.0,79,0.4,20,6.1,67,103,170,5,0,153,148,152,4,1,2
2124,140,0.001,0.000,0.006,0.000,0.0,0.0,78,0.4,27,7.0,66,103,169,6,0,152,147,151,4,1,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          2126 non-null   int64  
 1   accelerations                                           2126 non-null   float64
 2   fetal_movement                                          2126 non-null   float64
 3   uterine_contractions                                    2126 non-null   float64
 4   light_decelerations                                     2126 non-null   float64
 5   severe_decelerations                                    2126 non-null   float64
 6   prolongued_decelerations                                2126 non-null   float64
 7   abnormal_short_term_variability                         2126 non-null   int64  
 8   mean_value_of_short_term_variability  

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [3]:
X = data.drop('fetal_health', axis=1)
y = data['fetal_health'] - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\jehad\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, verbose=1)
# print(y_train)
# print(pd.isnull(X_train_scaled).sum()) 
# print(pd.isnull(X).sum()) 
# print((X_train_scaled == float('inf')).sum()) 
# print((X_test_scaled == float('inf')).sum())
# Dense(3, activation='softmax') # Output layer for 3 classes (0, 1, 2)
history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=1)

Epoch 1/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6236 - loss: 3.4093 - val_accuracy: 0.8059 - val_loss: 0.5030
Epoch 2/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6677 - loss: 2.9238 - val_accuracy: 0.8765 - val_loss: 0.3732
Epoch 3/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6838 - loss: 2.3570 - val_accuracy: 0.8853 - val_loss: 0.3493
Epoch 4/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7399 - loss: 1.6705 - val_accuracy: 0.8912 - val_loss: 0.4225
Epoch 5/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7275 - loss: 1.5170 - val_accuracy: 0.8941 - val_loss: 0.4081
Epoch 6/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7344 - loss: 1.4062 - val_accuracy: 0.8971 - val_loss: 0.4409
Epoch 7/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7235 - loss: 1.3199 - val_accuracy: 0.8971 - val_loss: 0.4670
Epoch 8/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7523 - loss: 1.2039 - val_accuracy: 0.8941 - val_loss:

In [5]:
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.2f}")
# Generate predictions
y_pred = model.predict(X_test_scaled)
y_pred_classes = y_pred.argmax(axis=1)

Test Accuracy: 0.90
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [6]:
y_train_pred = model.predict(X_train_scaled)
y_train_pred_classes = y_train_pred.argmax(axis=1)

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step


In [7]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred_classes, target_names=['Normal', 'Suspect', 'Pathological'])
print(report)
print("training data")
report = classification_report(y_train, y_train_pred_classes, target_names=['Normal', 'Suspect', 'Pathological'])
print(report)
# Plot training history (optional)
import matplotlib.pyplot as plt


              precision    recall  f1-score   support

      Normal       0.95      0.95      0.95       333
     Suspect       0.68      0.70      0.69        64
Pathological       0.81      0.76      0.79        29

    accuracy                           0.90       426
   macro avg       0.82      0.80      0.81       426
weighted avg       0.90      0.90      0.90       426

training data
              precision    recall  f1-score   support

      Normal       0.95      0.96      0.96      1322
     Suspect       0.71      0.77      0.74       231
Pathological       0.91      0.71      0.80       147

    accuracy                           0.92      1700
   macro avg       0.86      0.81      0.83      1700
weighted avg       0.92      0.92      0.92      1700

